In [1]:
import aiida

# must execute it in the first place
aiida.load_profile()


In [2]:
from aiida.orm import Code
from aiida.orm import Str, Int, Dict, List, Float
from aiida.engine import calcfunction, WorkChain, ToContext, append_
from aiida.plugins import DataFactory, WorkflowFactory
from itertools import cycle
import os

from aiida.engine import calcfunction, workfunction, submit, run
from aiida.orm import load_code, load_node
from tools import wait_for_node_finished


In [3]:
# codeの定義を行う。
from tools.aiida_support import get_or_create_local_computer, get_or_create_code
from os.path import expanduser
import os
home = expanduser("~")
work_directory = os.path.join(home, 'aiida')
computer_local = get_or_create_local_computer(work_directory, 'localhost')
code = get_or_create_code('alamode.alm_suggest',
                          computer_local,
                          '/home/max/Documents/alamode/bin/alm')
code_alm_sugget = code

code_alm_sugget


<Code: Remote code 'alamode.alm_suggest' on localhost, pk: 59092, uuid: 8a33eb3c-cb85-476d-b4ec-fba4fbdbd5cc>

In [4]:
import json
with open("setting.json") as f:
    _dic = json.load(f)
print(_dic)
CWD = _dic["CWD"]
_path = os.path.abspath(CWD)
os.makedirs(_path, exist_ok=True)
_path


{'CWD': 'run29'}


'/home/max/Documents/aiida-test/alamode-aiida/example/run29'

In [5]:
from os.path import expanduser
from tools import NodeBank


# 再実行時に作成したノードを用いるためにnodebankを使う。
g_force = False
nodebank = NodeBank(_path, force=g_force)


In [6]:
# g_dispfile_folder = nodebank.load("dispfile_folder")
# g_displace_result = nodebank.load("displace_result")
g_superstructure = nodebank.load("superstructure")  # 長周期安定構造

g_norder = nodebank.dump('cubic_norder', Int(2))  # cubic term
g_prefix = nodebank.dump('cubic_prefix', Str("cubic"))  # cubic term prefix
g_norder, g_prefix


(<Int: uuid: 024283bc-0bab-4a13-be6c-06e64a7c70dc (pk: 59328) value: 2>,
 <Str: uuid: c712e126-400d-4846-9bbf-772c331c7f2b (pk: 59329) value: cubic>)

In [7]:
g_cutoff = Dict(dict={"Si-Si": {None, 7.3}})  # cubic term cutoff 定義

g_cutoff = nodebank.load_or_dump("cutoff2", g_cutoff)
g_cutoff


debug, node uuid: d79dd6c7-ed96-4fc2-8664-290dc12a838d (unstored)
debug, load None
debug, force dump


<Dict: uuid: d79dd6c7-ed96-4fc2-8664-290dc12a838d (pk: 59330)>

In [8]:
g_primstructure = nodebank.load("primstructure")  # primitive cell
g_diag = nodebank.load("diag")  # 長周期
g_almprefix = "_".join([g_primstructure.get_formula(),
                        "x".join(map(str, g_diag.get_array("diag"))),
                        g_prefix.value,
                        ])
print(g_almprefix)

g_wd_dos = Str(g_almprefix)
g_wd_dos


Ga2N2_1x1x1_cubic


<Str: uuid: 207148c5-b368-4806-b3b9-da3b911f9432 (unstored) value: Ga2N2_1x1x1_cubic>

In [9]:
g_root_wd = nodebank.load('root_wd')  # root directory
g_root_wd
g_cwd = Str(os.path.join(g_root_wd.value, g_wd_dos.value))
print(g_cwd)
g_cwd = nodebank.dump(f'{g_prefix.value}_cwd', g_cwd)  # cubic termの結果を置くsubdirectory
os.makedirs(g_cwd.value, exist_ok=True)


uuid: 7311e76c-dddc-474a-88ac-81014ccec5cb (unstored) value: /home/max/Documents/aiida-test/alamode-aiida/example/run29/Ga2N2_1x1x1_cubic


In [10]:
#codename = "alm_suggest@tutor"
#code = Code.get_from_string(codename)
code = code_alm_sugget

builder = code.get_builder()
builder.structure = g_superstructure  # 長周期構造
builder.prefix = g_prefix  # 名前付け
# builder.mode = Str("suggest")
builder.cwd = g_cwd  # 　保存directory
builder.norder = g_norder
builder.cutoff = g_cutoff

builder.metadata = {
    'options': {
        'resources': {'tot_num_mpiprocs': 1, 'num_machines': 1},
        'input_filename': "alm_suggest_cubic.in",
        'output_filename': "alm_suggest_cubic.out",
    }}

key = f"{g_prefix.value}_alm_suggest"

g_alm_suggest_future = nodebank.load(key, g_force)
if g_alm_suggest_future is None:
    # if True:
    g_alm_suggest_future = submit(builder)  # temporary run(), use submit()
    print(g_alm_suggest_future)
    wait_for_node_finished(g_alm_suggest_future, 5)
    if g_alm_suggest_future.is_finished_ok:
        nodebank.dump(key, g_alm_suggest_future)


uuid: c40c1f46-9c1b-44da-87c2-14ffa7f6d1fd (pk: 59334) (aiida.calculations:alamode.alm_suggest)
wait another 5 sec.
wait another 5 sec.
wait another 5 sec.
wait another 5 sec.
wait another 5 sec.
wait another 5 sec.
wait another 5 sec.
wait another 5 sec.
wait another 5 sec.
wait another 5 sec.
wait another 5 sec.
wait another 5 sec.
wait another 5 sec.
wait another 5 sec.
wait another 5 sec.
wait another 5 sec.
wait another 5 sec.
wait another 5 sec.
wait another 5 sec.
wait another 5 sec.
wait another 5 sec.
wait another 5 sec.
wait another 5 sec.
wait another 5 sec.
wait another 5 sec.
True True


In [11]:
g_alm_suggest_future.outputs.pattern_folder.list_object_names()


['cubic.pattern_ANHARM3', 'cubic.pattern_HARMONIC']

In [12]:
g_alm_suggest_future.outputs.results["disp_pattern"]


{'ANHARM3': 'cubic.pattern_ANHARM3', 'HARMONIC': 'cubic.pattern_HARMONIC'}

In [13]:
g_cubic_pattern_files = g_alm_suggest_future.outputs.results["disp_pattern"]["ANHARM3"]
g_cubic_pattern_files = {'pattern_files': [g_cubic_pattern_files]}
g_cubic_pattern_files


{'pattern_files': ['cubic.pattern_ANHARM3']}

In [14]:
nodebank.dump(f"{g_prefix.value}_suggest_result", g_alm_suggest_future.outputs.results)


<Dict: uuid: 4b9ec204-316a-4755-9dce-2d7691df4ee9 (pk: 59337)>

In [15]:
nodebank.dump(f'{g_prefix.value}_suggest_pattern_folder',
              g_alm_suggest_future.outputs.pattern_folder)


<FolderData: uuid: 843821a8-1924-41dd-a4f1-1d2af54e7843 (pk: 59338)>

In [16]:
g_alm_suggest_future.outputs.results.attributes

{'num_disp': {'ANHARM3': '610', 'HARMONIC': '10'},
 'disp_pattern': {'ANHARM3': 'cubic.pattern_ANHARM3',
  'HARMONIC': 'cubic.pattern_HARMONIC'},
 'num_free_fcs,': {'ANHARM3': 1678, 'HARMONIC': 66}}